In [ ]:
#Compute Trend with Seasonal Mann-Kendall Test

import pymannkendall_modified as mk

data = gws


p = []
slope = []
slope_lo = []
slope_up = []

# Iterate over all 1 degree cells
# and compute p-value, slope and upper and lower confidence interval of slope

for i in np.arange(len(data.lat.values)):
    for j in np.arange(len(data.lon.values)):

        try:
            p_x = mk.seasonal_test(data[:, i, j]).p
            slope_x = mk.seasonal_test(data[:, i, j]).slope
            slope_lo_x = mk.seasonal_test(data[:, i, j]).slope_lo
            slope_up_x = mk.seasonal_test(data[:, i, j]).slope_up

        except:
            p_x = np.nan
            slope_x = np.nan
            slope_lo_x = np.nan
            slope_up_x = np.nan

        p.append(p_x)
        slope.append(slope_x)
        slope_lo.append(slope_lo_x)
        slope_up.append(slope_up_x)


#Convert lists back to xarray datasets

temp = np.copy(p).reshape(data.lat.size, data.lon.size)
p = xr.DataArray(temp, dims=('lat', 'lon'), coords={'lat': data.lat, 'lon': data.lon}, name='p')

temp = np.copy(slope).reshape(data.lat.size, data.lon.size)
slope = xr.DataArray(temp, dims=('lat', 'lon'), coords={'lat': data.lat, 'lon': data.lon}, name='slope')

temp = np.copy(slope_lo).reshape(data.lat.size, data.lon.size)
slope_lo = xr.DataArray(temp, dims=('lat', 'lon'), coords={'lat': data.lat, 'lon': data.lon}, name='slope_lo')

temp = np.copy(slope_up).reshape(data.lat.size, data.lon.size)
slope_up = xr.DataArray(temp, dims=('lat', 'lon'), coords={'lat': data.lat, 'lon': data.lon}, name='slope_up')


# # plot Trend data
map_data = slope
map_data.plot(robust=True, cmap=plt.cm.RdBu, vmin=-20, vmax=20)
plt.xlabel('longitude')
plt.ylabel('latitude')
#plt.axes().set_aspect('equal')
plt.title('Trend in GWS')
plt.tight_layout()
plt.show()

# # plot Significance
map_data = p.where(p < 0.05)
map_data.plot(robust=True, cmap=plt.cm.viridis)
plt.xlabel('longitude')
plt.ylabel('latitude')
#plt.axes().set_aspect('equal')
plt.title('Sign. of Trend (p-val.)')
plt.tight_layout()
plt.show()


# Export all data to netcdf
p.to_netcdf("p_GRACE_GWS_with_ERA5.nc")
#slope.to_netcdf("slope_GRACE_GWS_with_ERA5.nc")